In [1]:
from pygel3d import hmesh, jupyter_display as jd, gl_display as gl, graph
import numpy as np
import math
from commons.utils import *
from commons.display import *
from medial_axis_processing import unfolding, inverse_apply, preprocessing
from medial_axis_processing.medial_axis import *
from scipy.spatial import KDTree

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
input_mesh = hmesh.load("data/hand.off")
# input_mesh = hmesh.load("data/voxelized.obj")

# jd.display(input_mesh)

In [4]:
file_path = "hand.ma___v_100___e_185___f_86.ma"
# file_path = "voxelized1.ma___v_500___e_1466___f_1003.ma"
vertices, edges, faces = read_qmat(file_path)

In [5]:
# file_path = "sim_MA___v_67___e_122___f_56.obj"
# vertices, edges, faces = read_ma_ca(file_path)

In [6]:
# to_medial_sheets_and_curves(vertices, edges, faces)

In [24]:
medial_sheet = to_medial_sheet(vertices, faces)
# medial_sheet = hmesh.load("results/hand_qmat.obj")
medial_curves, g = to_medial_curves(vertices, edges, faces)
medial_curves = []

In [25]:
jd.display(medial_sheet)

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array([  0,   3,  98, 149,  10,  11,  14,  75, 159,  98,  20,  23,  26,  27,
                           30,  29,   3, 103,  35,  98,  39,  42,  45,  48,  26,  60, 100,  56,
                           59,  60,  62,  63,  66,  67,  70,  71,  74,  75,  77, 106,  82,  83,
                           86, 156,  91,  94,  97,  92,  98,  26, 100, 102, 103, 105, 108, 106,
                          111,  39, 115,  10, 119, 122, 123, 126, 129,  39, 133, 136, 115, 140,
                           42, 144,  44, 148, 151, 149, 152, 155, 156, 158, 159, 161, 164, 167,
                           82,  86]),
              'j': array([  1,   4,   6,   8,  11,  13,  15,  14,  17,   7,  21,  24,  27,  29,
                           31,  33,   5,  35,  37,  19,  40,  43,  46,  49,  28,  52,  54,  57,
                           60,  53,  63,  65,  67,  69,  71,  73,  75,  16,  78,  80,  83,  85,
             

In [26]:
trim = trimesh.Trimesh(vertices, faces)

connected_components = list(trim.split(only_watertight=False))
connected_components.sort(key=lambda x: -len(x.faces))
to_keep = connected_components[0]
medial_sheet = trimesh_to_manifold(to_keep)

medial_sheet = preprocessing.fix_normals(medial_sheet)

In [27]:
# jd.display(medial_sheet)

In [28]:
correspondences = build_ball_correspondences(input_mesh, vertices, start=0.005, step=0.001)

Chosen minimum gamma: 0.016000000000000007


In [29]:
medial_axis = MedialAxis(input_mesh, vertices, medial_sheet, medial_curves, correspondences)

In [30]:
display_medial_axis(medial_axis)

In [32]:
unfolded_sheet = unfolding.get_unfolded_sheet_positions(medial_axis)
unfolded_curves = unfolding.get_unfolded_curve_positions(medial_axis, medial_sheet.positions(), unfolded_sheet)

In [33]:
updated_sheet = hmesh.Manifold(medial_sheet)
updated_sheet.positions()[:] = unfolded_sheet
hmesh.flip_orientation(updated_sheet)

In [34]:
inverse_apply.apply_inverse_medial_axis_transform(medial_axis, unfolded_sheet, unfolded_curves)

In [35]:
jd.display(medial_axis.surface)

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array([ 116,  119,  116, ..., 6187, 6173, 6190]),
              'j': array([   0,    1,    1, ..., 6190, 6190, 6189]),
              'k': array([ 144,  116,    0, ..., 6188, 6187, 6188]),
              'type': 'mesh3d',
              'uid': 'dfc1077c-ae03-4e85-8cdd-99840d48db7a',
              'x': array([-0.2005817 , -0.21093102, -0.22222295, ...,  0.16874349,  0.15686188,
                           0.16775826]),
              'y': array([ 0.11347763,  0.1088319 ,  0.10265707, ..., -0.77391483, -0.77867602,
                          -0.76792776]),
              'z': array([ 0.037186  ,  0.03265271,  0.02840685, ..., -0.08717659, -0.08568032,
                          -0.07825839])},
             {'hoverinfo': 'none',
              'line': {'color': 'rgb(125,0,0)', 'width': 1},
              'mode': 'lines',
              'type': 'scatter3d',
              'uid': 'f313960b-391e-4b4d-